In [2]:
from openai import OpenAI

## Create Assisstant

In [5]:
client = OpenAI()

assistant = client.beta.assistants.create(
  name="Legal_Assistant",
  instructions="You are a personal legal tutor. You interpret the legal documents based on the instructions provided.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [15]:
assistant.id

'asst_rCvQci8VClxoFwnUKgrdJUE4'

## Create Vector Store

In [6]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Legal Documents")
file_paths = ["data/raw/12 CFR Part 211 (up to date as of 2-07-2025).pdf", "data/raw/12 CFR Part 223 (up to date as of 2-07-2025).pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [7]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [17]:
assistant.id

'asst_rCvQci8VClxoFwnUKgrdJUE4'

## Query Question

In [14]:
from IPython.display import Markdown

instruction_prompt = '''
                ### Instruction:
                1. Provide a one-line answer to the user's question in clear, non-legal language.
                2. Create a table with three columns:
                   - **First Column:** Explain the legal interpretation in simple, plain text without legal jargon.
                   - **Second Column:** Translate the text into clear actionable business requirements in bullet points.
                   - **Third Column:** Provide three bullet points explaining requirements, permissions, and prohibitions in clear language.
                3. Double-check the format to ensure it meets these specifications.
                4. Ensure the language is simple, precise, and free of complex terminology.

                **Table Format Example:**

                | Legal Interpretation (Plain Text) | Business Requirements (Simple Terms) | Key Points (Requirements, Permissions, Prohibitions) |
                |------------------------------------|--------------------------------------|-----------------------------------------------------|
                | [Explain legal concept simply]     | [Translate into actionable business requirements in bullet points] | - **Requirement:** [What must be done]  <br> - **Permission:** [What is allowed]  <br> - **Prohibition:** [What is not allowed] |
                    '''

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": instruction_prompt
        },
        {
            "role": "user",
            "content": "How do you monitor compliance with supervisory reporting requirements under § 211.13?"
        }
    ]
)
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

output_text = f"**Response:**\n\n{message_content.value}\n\n"
if citations:
    output_text += "**Citations:**\n\n" + "\n".join(citations)

# Display in Markdown format
display(Markdown(output_text))

**Response:**

To monitor compliance with supervisory reporting requirements under § 211.13, organizations should maintain effective record systems, report risk asset and operation audits, ensure access to information for examiners, and file necessary reports to relevant authorities[0].

| Legal Interpretation (Plain Text) | Business Requirements (Simple Terms) | Key Points (Requirements, Permissions, Prohibitions) |
|------------------------------------|--------------------------------------|-----------------------------------------------------|
| Banks must keep track of foreign branch activities and report to authorities. | - Set up records and control systems for activities.<br>- Regularly report on financial risks and audits.<br>- Share requested information with examiners. | - **Requirement:** Ensure accurate reporting on foreign operations.<br> - **Permission:** Maintain joint venture transaction records.<br> - **Prohibition:** Do not withhold required information from examiners. |

**Citations:**

[0] 12 CFR Part 211 (up to date as of 2-07-2025).pdf